<a href="https://colab.research.google.com/github/GuillermoSego/TSF/blob/main/Open_w_Memoria_dinamcia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memoria dinámica

Utilizamos la memoria dinámica cuando no sabemos el tamaño del archivo que estamos utillizando. Para utilizar la memoria hay varios pasos:

* Reservar la memoria
* Usar la memoria 
* Cerrar la memoria

La biblioteca que contiene el manejo de la memoria dinámica se llama *stdlib.h*. Necesitamos varias rutinas para hacer el programa

1. **malloc** Tamaño del buffer de datos en bytes
2. **realloc** Hace crecer la memroia
3. **free** Liberar la memoria

Necesitamos un **apuntador doble** para poder manejar la memoria. 

In [ ]:
// Memoria dinamica

#include <stdio.h>
#include <stdlib.h>

#define SIZE 131

/*
    Para checar cuando se nos acaba la memoria se hace con un contador, se hace
    un ciclo descendente y cuando se termina de leer el ciclo el pograma se sale. 
    Hacemos con un while(1)
*/

void ErrorManagement(int e)
{
    if (e == 0) {
        
        printf("Error al abrir el archivo\n");
    }
    
    if (e == 1) {
        
        printf("Error al intentar reservar memoria");
    }

}

// Esta funcion llena un arreglo
void Full(double* Ptr)
{
    
    int n = SIZE, i = 0;
    
    do {
        
        *Ptr = i;
        
        if (i == (SIZE)){
            Ptr = realloc(Ptr, 2 * 1024); 
        }
        
        Ptr ++;
        
        i ++;
        
    } while(i<n);  
    
}

// Imprimir un arreglo
void Imp(double* Ptr){
    
    int n = SIZE, i = 0;
    
    do {
        
        printf("%lf\n", *Ptr);
        Ptr ++;
        
        
        i ++;
        
    } while(i<n);
    
}

int main()
{
    
    /*
    Con la función malloc reservamos un bloque de memoria, devuelve un apuntador 
    al inicio de la misma 
    */
    
    double* vPtr;
    
    /*
    Reservamos 1024 bytes porque ese es el tamaño de una pagina de 
    codigo en Windows
    */
    vPtr = malloc(1024);
    
    
    // Manejador de errores
    if (vPtr == NULL){
        ErrorManagement(1);
        return 0;
    }
    
    Full(vPtr);
    
    
    Imp(vPtr);
    
    // Liberamos la memoria, es muy importante liberar la memoria
    free(vPtr);
    


    return 0;
}

# Open CSV con memoria dinámica
Vamos a implementar el programa de *Open CSV* utilizando memoria dinámica. 

In [ ]:
%%writefile O.c

// Manejador de errores
void ErrorManagement(int e)
{
    if (e == 0) {
        
        printf("Error al abrir el archivo\n");
    }

}

// Esta rutina abre un archivo CSV y lo devuelve a un vector
int* OpenCSV(char* arg, double DatV[BF])
{
    
  // Declaramos una variable tipo FILE
  FILE* Doc;

  // Abrimos el archivo. "r": Abrir un archivo existente para leer. 
  // Hacemos que el apuntador Doc apunte en la direccion de los datos
  Doc = fopen ( arg, "r" );
 
  // Manejamos el error: no se pueda abrir el archivo
  if (Doc == NULL)
  {
      ErrorManagement( 0 );
      return 0;
  } 
 
  // Continuamos con la estructura para leer los datos
 
  // Primero creamos un buffer para guardar los datos
	char buffer[BF];

  // Se lee la primera linea y se almacena en el buffer como variable char
	fgets(buffer, BF, Doc);
  
  /* 
 
    En este punto todo esta almacenado en el buffer como una sola cadena de 
    caracteres. Por lo que para poder acceso a los datos se tiene que convertir
    esta cadena a binario, separando por comas "," ya que un archivo tipo csv
    separa los datos por comas.
 
  */
  
  // Creamos un apuntador que apunte a la cadena de caracteres
  char* BfPtr = buffer;
 
  // Calculamos el número de columnas contando las comas de la primera linea
  int column = 0;
    
  do{
      column += ( *BfPtr == ',' );

  } while( *BfPtr ++);
 
  // Convertimos a binario
 
  // Apuntamos en la direccion del buffer
  BfPtr = buffer;
 
  // Apuntador al vector de datos
  double* DatPtr = DatV;
  
  // Creamos un contador igual al número de columnas
  // Declaramos el parámetro adicional para la función sscanf
  int i = column, length;
 
  // Leemos el primer renglon 

  do{
        
      BfPtr += length;
      sscanf( BfPtr, "%lf,%n", DatPtr ++, &length );
        
    } while(i--);

  /*
  
    En este punto en el vector DatV tenemos guardado la primer fila de los datos.
    Ahora tenemos que guardar las siguientes filas.
 
    El apuntador del buffer BfPtr se encuentra apuntando hacia el primer dato 
    de la última columna, para asegurarnos de que apuntamos exactamente al ultimo
    elemento de la primera fila, le sumamos la longitud length

  */
  BfPtr += length;

  /*
 
  Leemos las siguientes lineas del buffer y las ponemos en el apuntador 
  de los datos

  */
  sscanf( BfPtr, "%lf", DatPtr); 

  // Recorremos el apuntador de los datos
  i = column;
  DatPtr = DatV;

	do {
		*DatPtr ++;
	} while( i-- );
		
	// Se termina de leer el archivo
	
  /*
 
  En este momento leemos los demas datos del apuntador Doc, y los guardamos en
  el apuntador de los datos DatPtr, sin embargo, tenemos que llegar al final de
  los datos. Esto lo hacemos creando una variable llamada dummy en la que guar-
  damos el momento en el que se llega al final, es decir cuando tenemos que los 
  datos tienen la forma "%lf%c"

 */
  
  int Error = 0, col = column+1; 
  char dummy;

	do{

		Error = fscanf(Doc, "%lf%c", DatPtr ++, &dummy);
         
	} while(Error > 0);


  // Cerramos el archivo, es muy importante cerrar el archivo
  fclose ( Doc );
 
  static int size[2];
  size[0] = (DatPtr - DatV)/ (col);
  size[1] = col;
 
  return size;

}

int main(int nArg, char *arg[])
{

    /*
 
      Declaramos un vector en el que vamos a guardar los datos.
      Le apuntamos dos veces.
 
    */
    double VecM[BF]; 
 
    // Creamos un vector en el que vamos a guardar los valores de N y M
    int size[2];
    int* sizePtr = size;

    // Abrimos el archivo y guardamos en el vector DatM los datos
    sizePtr = OpenCSV(*(arg+1), VecM);
 
    int SA = *sizePtr, ST = *(sizePtr+1);
 
    // Declaramos un nuevo vector en el que guardaremos la transpuesta 
    double DatM[ST*SA];
    double* DatTPtr = DatM;
    
    // Determinamos la transpuesta
    MatrixT(SA, ST, VecM, DatTPtr);
 
    
    
    return 0;
}